In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd

import requests
import json
from datetime import datetime

# Prepair shapefile data
You need to prepair a shapefile of your the regions you want to download. It may have many columns, but the most important and must-appear is geometry column.

In [2]:
mekongReservoirs = gpd.read_file('mekongReservoirs')

In [3]:
mekongReservoirs.columns

Index(['Lake_name', 'Country', 'Lake_area', 'geometry'], dtype='object')

In [4]:
top3LargestReservoirs = mekongReservoirs.iloc[:3]

In [8]:
top10LargestReservoirs = mekongReservoirs.iloc[:10]

In [9]:
top3To10LargestReservoirs = mekongReservoirs.iloc[3:10]

# GetModisData class
A class for easy download a subset of modis data. You only have to learn how to use the method:  
`GetModisData.createOrderURLURL(reservoirsGeoDataFrame, modisProductCode, email, recreateDownloadFile)`.

In [10]:
class GetModisData:
    url = "https://modis.ornl.gov/rst/api/v1/"
    header = {'Accept': 'application/json'} # Use following for a csv response: header = {'Accept': 'text/csv'}
    subsetUrl = 'https://modis.ornl.gov/subsetdata/'
    
    def calDistanceInKm(point1, point2):
        import pyproj
        geod = pyproj.Geod(ellps='WGS84')
        angle1, angle2, distance = geod.inv(point1.x, point1.y, point2.x, point2.y)
        return distance/1000

    
    def findBorderRect_1(polygon):
        from shapely.geometry import Point
        xmin, ymin, xmax, ymax = polygon.bounds

        topLeft = Point(xmin, ymax)
        bottomLeft = Point(xmin, ymin)
        topRight = Point(xmax, ymax)
        bottomRight = Point(xmax, ymin)
        vertices = [topLeft, topRight, bottomLeft, bottomRight]

        centroid = polygon.centroid
        kmAboveBelow = kmLeftRight \
                     = np.int32(np.ceil(max([GetModisData.calDistanceInKm(centroid, v) for v in vertices])))
        return centroid, kmAboveBelow, kmLeftRight
    
    
    def getMODISDate(reservoirRowDf):
        # Submit request
        row = reservoirRowDf
        response = requests.get(GetModisData.url
                                + row['product'] 
                                + '/dates?latitude=' + str(row['latitude']) 
                                + '&longitude='+ str(row['longitude']), headers=GetModisData.header)

        # Get dates object as list of python dictionaries
        dates = json.loads(response.text)['dates'] 

        # Convert to list of tuples; change calendar_date key values to datetimes
        dates = [(datetime.strptime(date['calendar_date'], "%Y-%m-%d"), date['modis_date']) for date in dates]

        # Get MODIS dates nearest to start_date and end_date and add to new pandas columns
        start_MODIS_date = min(date[1] for date in dates if date[0] > row['start_date'])
        end_MODIS_date = max(date[1] for date in dates if date[0] < row['end_date'])

        return start_MODIS_date, end_MODIS_date
    
    
    def createReservoirsDownloadDataframe(reservoirsGeoDataFrame, productCode, email):
        df = pd.DataFrame(columns=['site_id', 'product', 'latitude', 'longitude', 
                                   'email', 'start_date', 'end_date',
                                   'kmAboveBelow', 'kmLeftRight', 'order_uid'])
        
        default_start_date = pd.to_datetime('2000-01-01')
        default_end_date = pd.to_datetime('2018-07-25')
        
        for index, reservoirRow in reservoirsGeoDataFrame.iterrows():
            center, kmAboveBelow, kmLeftRight = GetModisData.findBorderRect_1(reservoirRow.geometry)
            row = {'site_id': 'site{}'.format(index),
                    'product': productCode,
                    'latitude': center.y,
                    'longitude': center.x,
                    'email': email,
                    'start_date': default_start_date,
                    'end_date': default_end_date,
                    'kmAboveBelow': kmAboveBelow,
                    'kmLeftRight': kmLeftRight,
                    'order_uid': 'None'}
            
            row['start_date'], row['end_date'] = GetModisData.getMODISDate(row)
            df = df.append(row, ignore_index=True)
        return df
    
    
    def createOrderURL_1(reservoirsDownloadDf, downloadFileResult=None):
        # Make list to collect order UIDs
        order_uids = []
        productCode = reservoirsDownloadDf.loc[0, 'product']
        
        for index, row in reservoirsDownloadDf.iterrows():
            if row['order_uid'] == 'None':
                # Build request URL
                requestURL = GetModisData.url + productCode \
                                              + "/subsetOrder?latitude=" + str(row['latitude']) \
                                              + "&longitude=" + str(row['longitude']) \
                                              + "&email=" + row['email'] \
                                              + "&uid=" + row['site_id'] \
                                              + "&startDate=" + row['start_date'] \
                                              + "&endDate=" + row['end_date'] \
                                              + "&kmAboveBelow=" + str(row['kmAboveBelow']) \
                                              + "&kmLeftRight=" + str(row['kmLeftRight'])

                print(requestURL)
                # Submit request
                response = requests.get(requestURL, headers=GetModisData.header)
                try:
                    order_uid = GetModisData.subsetUrl + json.loads(response.text)['order_id']
                except TypeError as err:
                    print(response.text)
                    break

                # Append UID to list
                order_uids.append(order_uid)
                reservoirsDownloadDf.loc[index, 'order_uid'] = order_uid

        print(order_uids)
        if downloadFileResult is None:
            reservoirsDownloadDf.to_csv(productCode + '.csv')
        else:
            reservoirsDownloadDf.to_csv(downloadFileResult + '.csv')
        return order_uids
    
    
    def createOrderURL(reservoirsGeoDataFrame, modisProductCode='MOD13Q1', 
                       email='lamductan@gmail.com', recreateDownloadFile=False,
                       downloadFileResult=None):
        import os
        if downloadFileResult is None:
            downloadFileResult = modisProductCode
        if not os.path.exists(downloadFileResult + '.csv') or recreateDownloadFile:
            reservoirsDownloadDf = GetModisData.createReservoirsDownloadDataframe(reservoirsGeoDataFrame, 
                                                                                modisProductCode, email)
            reservoirsDownloadDf.to_csv(downloadFileResult + '.csv')
        else:
            reservoirsDownloadDf = pd.read_csv(downloadFileResult + '.csv')
        return GetModisData.createOrderURL_1(reservoirsDownloadDf, downloadFileResult)

# Example how to use GetModisClass

In [11]:
downloadTerraUrlList = GetModisData.createOrderURL(reservoirsGeoDataFrame=top3To10LargestReservoirs, 
                                                  modisProductCode='MOD13Q1',
                                                  email='1512489@student.hcmus.edu.vn',
                                                  recreateDownloadFile=False,
                                                  downloadFileResult='3-7_MOD13Q1')

https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=16.729528722980266&longitude=102.53383782042863&email=1512489@student.hcmus.edu.vn&uid=site3&startDate=A2000049&endDate=A2018177&kmAboveBelow=31&kmLeftRight=31
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=16.748836915920027&longitude=103.43834851186996&email=1512489@student.hcmus.edu.vn&uid=site4&startDate=A2000049&endDate=A2018177&kmAboveBelow=37&kmLeftRight=37
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=15.044525697834212&longitude=105.3816076514864&email=1512489@student.hcmus.edu.vn&uid=site5&startDate=A2000049&endDate=A2018177&kmAboveBelow=29&kmLeftRight=29
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=24.699463586321087&longitude=100.34201766875154&email=1512489@student.hcmus.edu.vn&uid=site6&startDate=A2000049&endDate=A2018177&kmAboveBelow=27&kmLeftRight=27
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=19.169775127740934&longitude=99.87569501737315

In [12]:
downloadAquaUrlList = GetModisData.createOrderURL(reservoirsGeoDataFrame=top10LargestReservoirs, 
                                                  modisProductCode='MYD13Q1',
                                                  email='1512489@student.hcmus.edu.vn',
                                                  recreateDownloadFile=False)

https://modis.ornl.gov/rst/api/v1/MYD13Q1/subsetOrder?latitude=12.909667465035685&longitude=104.04910517189141&email=1512489@student.hcmus.edu.vn&uid=site0&startDate=A2002185&endDate=A2018185&kmAboveBelow=64&kmLeftRight=64
https://modis.ornl.gov/rst/api/v1/MYD13Q1/subsetOrder?latitude=25.78871606370018&longitude=100.1953318267777&email=1512489@student.hcmus.edu.vn&uid=site1&startDate=A2002185&endDate=A2018169&kmAboveBelow=24&kmLeftRight=24
https://modis.ornl.gov/rst/api/v1/MYD13Q1/subsetOrder?latitude=18.586321530458974&longitude=102.63476005462799&email=1512489@student.hcmus.edu.vn&uid=site2&startDate=A2002185&endDate=A2018169&kmAboveBelow=37&kmLeftRight=37
https://modis.ornl.gov/rst/api/v1/MYD13Q1/subsetOrder?latitude=16.729528722980266&longitude=102.53383782042863&email=1512489@student.hcmus.edu.vn&uid=site3&startDate=A2002185&endDate=A2018169&kmAboveBelow=31&kmLeftRight=31
https://modis.ornl.gov/rst/api/v1/MYD13Q1/subsetOrder?latitude=16.748836915920027&longitude=103.43834851186996

In [14]:
downloadTerraUrlList = GetModisData.createOrderURL(reservoirsGeoDataFrame=top3LargestReservoirs, 
                                                  modisProductCode='MOD13Q1',
                                                  email='1512489@student.hcmus.edu.vn',
                                                  recreateDownloadFile=False,
                                                  downloadFileResult='MOD13Q1_1')

https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=12.909667465035685&longitude=104.04910517189141&email=1512489@student.hcmus.edu.vn&uid=site0&startDate=A2000049&endDate=A2018177&kmAboveBelow=64&kmLeftRight=64
https://modis.ornl.gov/rst/api/v1/MOD13Q1/subsetOrder?latitude=25.78871606370018&longitude=100.1953318267777&email=1512489@student.hcmus.edu.vn&uid=site1&startDate=A2000049&endDate=A2018177&kmAboveBelow=24&kmLeftRight=24
"1512489@student.hcmus.edu.vn has placed 21 orders today. Cannot place more than 20 orders a day through the Web Service."

['https://modis.ornl.gov/subsetdata/27Jul2018_10:15:33_567417866L12.909667465L104.049105172S513L513_MOD13Q1_site0']
